# Vehicle Routing Problem
[Murwan Siddig](mailto:msiddig@clemson.edu)

------------------------------------------------------------------------
## Two Index Formulation

In [1]:
include("data_VRP.jl");

In [2]:
#technical lines
using Random;
using Distributions;
using LinearAlgebra
using JuMP;
using Gurobi;
const GRB_ENV = Gurobi.Env();


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [3]:
#Given a solution X for the VRP, this function does a graph search on the graph induced by V & E' where Xe=1 
function bfs(s,x,n)
    graph = [];
    for i in 1:nn
        tempList = []
        for j in 1:nn
            if x[i,j]>=1-1e-6
                push!(tempList,j);
            end
        end
        push!(graph,tempList);
    end
    # s: index of the node where you start the graph search
    # graph: a representation of the graph using the adjacency list
    n = length(graph);
    visited = zeros(n);
    OPEN = Any[];
    push!(OPEN, s);
    while (length(OPEN) > 0)
        t = OPEN[1]; 
        visited[t] = 1;
        for i in graph[t]
            if visited[i] == 0
                push!(OPEN, i);
            end
        end
        filter!(e->e!=t, OPEN);
    end
    visitedNodeSet = Any[];
    for i=1:n
        if visited[i] == 1
            push!(visitedNodeSet, i);
        end
    end
    return visitedNodeSet;
end

bfs (generic function with 1 method)

In [4]:
#Function to extract the tour for each vehicle in operation
function extractTour(n, sol)
    #Initilaize the an empty list for the lists of tours
    tours = [];
    #The number of vehicles in operation is equal to the number of arcs leaving the depot node 
    used_vehilces = 0
    for j=1:nn
        if sol[1,j] >= 1-1e-6
            used_vehilces +=1
        end
    end
    count = used_vehilces
    #Keep doing this loop until the tour for each vehicle in use is obtained
    while count !=0
        count -=1
        tour = [1]  # Start at city 1 always
        cur_city = 1
        while true
            # Look for first arc out of current city
            for j = 1:n
                if sol[cur_city,j] >= 1-1e-6
                    # Found next city
                    push!(tour, j)
                    # Don't ever use this arc again
                 
                    sol[cur_city, j] = 0.0
                    sol[j, cur_city] = 0.0
                    # Move to next city
                    cur_city = j
                    break
                end
            end
            # If we have come back to 1, stop
            if cur_city == 1
                break
            end
        end  # end while
        push!(tours, tour)
    end
    return tours
end

extractTour (generic function with 1 method)

In [5]:
#Two-index formualation 
#-------------------------------------

# solveVRP
# Given a matrix of city locations, solve the VRP
# Inputs:
# nn       Number of cities
# c  n-by-2 matrix of (x,y) city locations
# Q Capacity of each vehicels 
# nv number of vehicles available 
# q the demand for each customer 
# Output:
# Tours:    Vector of vectors with order of cities visited for each vehicles 

function solveVRP(nn, Q, nv, q, c)
    
    # Create a model that will use Gurobi to solve
    VRP1 = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
    
    # x[i,j] is 1 iff we travel between i and j, 0 otherwise
    @variable(VRP1, x[1:nn,1:nn], Bin); 
    
    # Minimize length of each tour 
    @objective(VRP1, Min, sum(c[i,j]*x[i,j] for i=1:nn for j=1:nn));
    
    # Don't allow self-arcs
    # either i,j is chosen or j,i
    for i = 1:nn
        @constraint(VRP1, x[i,i] == 0)
        for j=1:nn
            @constraint(VRP1, x[i,j]+x[j,i] <= 1)
        end
    end  
    
                    
    #The number of vehicles leaving the depot equals the number of cars entering the depot which is also <= Number of availble vehicles 
                    
    @constraint(VRP1, sum(x[j,1] for j=2:nn)==sum(x[1,j] for j=2:nn));
    @constraint(VRP1, sum(x[j,1] for j=2:nn) <=nv);
    @constraint(VRP1, sum(x[1,j] for j=2:nn) <=nv);
    
    # The number of times you enter a city equals the number of times you leave it 
    # We must enter and leave every city once and only once
    for i=2:nn
        ind = [];
        for k=1:nn
            if k !=i
                push!(ind,k)
            end
        end
        @constraint(VRP1, sum(x[i,j] for j in ind)==sum(x[j,i] for j in ind));
        @constraint(VRP1, sum(x[i,j] for j in ind)==1);
        @constraint(VRP1, sum(x[j,i] for j in ind)==1);
    end
    cb_calls = Cint[]
    function violated(cb_data, cb_where::Cint)
        # You can reference variables outside the function as normal
        push!(cb_calls, cb_where)
        # You can select where the callback is run
        if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
            return
        end
        # You can query a callback attribute using GRBcbget
        if cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cint}()
            GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
            if resultP[] != GRB_OPTIMAL
                return  # Solution is something other than optimal.
            end
        end
        
        Gurobi.load_callback_variable_primal(cb_data, cb_where)
        xval = zeros(nn,nn);
        for ii=1:nn, jj=1:nn
            xval[ii,jj] = callback_value(cb_data, x[ii,jj])
        end
        visitedNodeSet = bfs(1,xval,nn)
        
        count = nn;
        
        #println("----\nInside subtour callback")
        if length(visitedNodeSet) < nn
            println("visitedNodeSet=",visitedNodeSet)
            println("length of visitedNodeSet=",length(visitedNodeSet))
            println("Graph is disconnected => This solution is infeasible")
            println("Adding subtour elimination cut")
            println("------------------------------------")
            arcs_from_violated = zero(AffExpr)
            ind = [];
            ind1 = [1];
            for k=2:nn
                if k in visitedNodeSet
                    push!(ind1,k)
                else
                    push!(ind,k)
                end
            end 
            arcs_from_violated = @build_constraint(sum(x[i,j] for i in ind1 for j in ind)>=cld(sum(q[j] for j in ind),Q))
            MOI.submit(VRP1, MOI.LazyConstraint(cb_data), arcs_from_violated)
        elseif length(visitedNodeSet) == nn
            #println("length of visitedNodeSet=",length(visitedNodeSet))
            println("Graph is connected")
            #println("We will extract the tours given by the current solution & check if it's feasible=> Check if the vehicle can contain all the demand of customers in the tour")
            tours = extractTour(nn, xval)
            count = length(tours)
            println("Tours=",tours)
            for i=1:length(tours)                                    
                ind = [1];
                ind1 = [];
                for k=2:nn
                    if k in tours[i]
                        push!(ind1,k)
                    else
                        push!(ind,k)
                    end
                end 
                if cld(sum(q[j] for j in ind1),Q) > 1
                    arcs_from_violated1 = @build_constraint(sum(x[k,j] for k in ind for j in ind1)>=cld(sum(q[j] for j in ind1),Q))
                    MOI.submit(VRP1, MOI.LazyConstraint(cb_data), arcs_from_violated1)
                    
                    arcs_from_violated1 = zero(AffExpr)
                    arcs_from_violated1 = sum(x[k,j] for k in ind for j in ind1)
                    println("Tour which is violating is=",i)
                    println("The number of vehicles being sent=", sum(xval[k,j] for k in ind for j in ind1))
                    println("The number of vehicles which needs to be sent=", cld(sum(q[j] for j in tours[i]),Q))
                    println("Adding subtour elimination cut")
                else
                    count -=1;
                end
            end
            #println("Number of violated tours=", count)
            if count == 0 
                println("No constraint is violated")
                return
            end
            println("------------------------------------")
        end                                                 
    end

    MOI.set(VRP1, MOI.RawParameter("LazyConstraints"), 1)
    MOI.set(VRP1, Gurobi.CallbackFunction(), violated)
    
    optimize!(VRP1)
    visitedNodeSet = bfs(1,value.(x),nn)
    println("***********************************************")
    println("***********************************************")
    println(" solved! ")
    println("===============================================")
    println("length of visitedNodeSet=",length(visitedNodeSet))
    println("Graph is connected")
    tours = extractTour(nn, value.(x))
    println("Tours = ",tours)
    println("This solution is Feasible and Optimal")
    
    return extractTour(nn, value.(x)), objective_value(VRP1), value.(x);
end

solveVRP (generic function with 1 method)

In [6]:
@time VRP, obj, xval = solveVRP(nn, Q, nv, q, c)
println("Optimal objective value = ", obj);

visitedNodeSet=Any[1, 4, 5, 7, 8, 9, 10, 12, 13, 16, 17, 20, 24, 26, 27, 28, 31]
length of visitedNodeSet=17
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1]
length of visitedNodeSet=1
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 3, 4, 5, 6, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
length of visitedNodeSet=16
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 21, 24, 25, 26, 27, 28, 29, 30, 31]
length of visitedNodeSet=24
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 26, 27, 28, 30, 31]
length of visited

visitedNodeSet=Any[1, 6, 11, 15, 17, 25, 26, 27]
length of visitedNodeSet=8
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 15, 27, 30, 31]
length of visitedNodeSet=5
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 11, 15, 17, 25, 26, 27]
length of visitedNodeSet=8
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 15, 27, 30, 31]
length of visitedNodeSet=5
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 8, 15, 17, 27, 31]
length of visitedNodeSet=7
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 7, 11, 15, 17, 25

Tour which is violating is=4
The number of vehicles being sent=0.0
The number of vehicles which needs to be sent=2
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 7, 11, 18, 20, 21, 22, 26, 30, 31, 32]
length of visitedNodeSet=12
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 7, 11, 18, 20, 21, 22, 26, 30, 31, 32]
length of visitedNodeSet=12
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 7, 11, 15, 18, 20, 21, 22, 25, 26, 30, 31, 32]
length of visitedNodeSet=14
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 7, 15, 18, 20, 21, 22, 25, 26, 30, 31, 32]
length of visitedNodeSet=13
Graph is disconnected => This solution is infeasible
Adding subtour elimin

Graph is connected
Tours=Any[[1, 15, 25, 1], [1, 17, 8, 14, 18, 20, 32, 22, 2, 13, 1], [1, 21, 6, 26, 11, 30, 16, 23, 10, 9, 12, 5, 29, 24, 3, 4, 7, 1], [1, 27, 31, 1], [1, 28, 19, 1]]
Tour which is violating is=2
The number of vehicles being sent=0.0
The number of vehicles which needs to be sent=2
Adding subtour elimination cut
Tour which is violating is=3
The number of vehicles being sent=0.0
The number of vehicles which needs to be sent=2
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 15, 19, 21, 27, 31]
length of visitedNodeSet=6
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 27, 30, 31]
length of visitedNodeSet=5
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 27, 30]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasibl

visitedNodeSet=Any[1, 21, 27]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 21, 27]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 13, 14, 17, 18, 20, 21, 22, 27, 32]
length of visitedNodeSet=11
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 21, 27]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 21, 27]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 3, 4, 7, 13, 27, 30]
length of visitedNodeSet=7
Graph

...

visitedNodeSet=Any[1, 27, 29]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 27, 29]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 6, 15, 21, 27]
length of visitedNodeSet=5
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 13, 14, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 13, 14, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 13, 14, 31]
length of visitedNodeSet=4
Graph is disconnected => This solu

visitedNodeSet=Any[1, 7, 13, 15, 31]
length of visitedNodeSet=5
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 13, 15, 31]
length of visitedNodeSet=5
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 15, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 15, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 7, 15, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 15, 31]
length of visitedNodeSet=3
Graph is disconnected => This 

Graph is connected
Tours=Any[[1, 13, 2, 14, 8, 27, 1], [1, 15, 25, 1], [1, 17, 31, 1], [1, 21, 6, 30, 26, 11, 16, 23, 10, 12, 5, 3, 18, 20, 32, 22, 1], [1, 28, 19, 9, 29, 24, 4, 7, 1]]
Tour which is violating is=4
The number of vehicles being sent=0.0
The number of vehicles which needs to be sent=3
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 6, 13, 21, 31]
length of visitedNodeSet=6
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 13, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 13, 31]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 15, 25, 30, 31]
length of visitedNodeSet=5
Graph is disconne

visitedNodeSet=Any[1, 28]
length of visitedNodeSet=2
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 27, 28]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 27, 28]
length of visitedNodeSet=3
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 13, 17, 18, 20, 22, 32]
length of visitedNodeSet=8
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 2, 7, 13, 15, 17, 21, 25, 28, 30, 31]
length of visitedNodeSet=11
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 8, 25, 30]
length of visitedNodeSet=4
Grap

visitedNodeSet=Any[1, 31]
length of visitedNodeSet=2
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
Graph is connected
Tours=Any[[1, 6, 26, 11, 16, 23, 10, 19, 9, 12, 5, 29, 7, 27, 1], [1, 8, 14, 22, 32, 20, 18, 24, 4, 3, 1], [1, 13, 2, 17, 31, 1], [1, 21, 30, 28, 1], [1, 25, 15, 1]]
Tour which is violating is=1
The number of vehicles being sent=0.0
The number of vehicles which needs to be sent=2
Adding subtour elimination cut
Tour which is violating is=2
The number of vehicles being sent=0.0
The number of vehicles which needs to be sent=2
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 27]
length of visitedNodeSet=2
Graph is disconnected => This solution is infeasible
Adding subtour elimination cut
------------------------------------
visitedNodeSet=Any[1, 15, 21, 27]
length of visitedNodeSet=4
Graph is disconnected => This solution is infeasible
Adding subtour eliminati